<a href="https://colab.research.google.com/github/zaedulislam/Deep_Learning_A-Z_Hands-On_Artificial_Neural_Networks/blob/main/Section%207%20-%20Building%20a%20CNN/Convolutional%20Neural%20Network%20(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network (CNN)

Notion Page: https://www.notion.so/zaedulislam/Deep-Learning-A-Z-Hands-On-Artificial-Neural-Networks-b0f4742181e74b8da987a0db61e0ced9#ea8f4602d93044f786949b38095a407c

### Importing the libraries

In [ ]:
# TensorFlow is already preinstalled as a library in Google Colab, but we still need to import it
import tensorflow as tf
# ImageDataGenerator class generate batches of tensor image data with real-time data augmentation
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.7.0'

## Part 1 - Data Processing

### Preprocessing the Training set

In [ ]:
# IMAGE AUGMENTATION
# "zoom_range" -> consists of zooming in or zooming out on the images
# "horizontal_flip" -> consists of flipping the images horizontally
# "shear_range" -> Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)

# FEATURE SCALING
# "rescale" -> This will apply feature scaling to each and every single one of your pixels by dividing their value by 255. Each pixel takes a value between zero and 255. 
# So by dividing all of them by 255, we indeed get all the pixel values between zero and one, which is just like a normalization. 
# Feature scaling is absolutely compulsory when training neural networks.

# train_datagen is an instance of ImageDataGenerator class
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


# "target_size" -> Final size of the images, that will be fed into the convolutional neural network. target_size=(150, 150) takes more time to train
# "batch_size" -> The size of the batches, meaning how many images we want to have in each batch. And the 32 is a classic default value.

training_set = train_datagen .flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

### Preprocessing the Test set

In [ ]:
# We're keeping the test data intact like the original ones by not applying any transformation. 
# However, we've to rescale their pixels to avoid information leakage from the test set. The future predict method of CNN will have
# to be applied to the same scaling as the one that was applied on the training set.
test_datagen = ImageDataGenerator(rescale=1./255) 

# "flow_from_directory(...)" access the dataset from the directory
# "target_size", "batch_size", "class_mode" should be same as for the "training_set"

test_test = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

## Part 2 - Building the CNN

### Initializing the CNN

In [ ]:
# CNN is also a sequence of layers as opposed to a Computational Graph

# keras -> library
# models -> module
# Sequential -> class

cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
# Adding the convolutional layer to the CNN so far initialized as a sequence of layers

# layers -> module
# Conv2D -> class

# relu -> rectifier activation function
# input_shape -> When we add the very first layer, whether it is a convolutional layer or a dense layer. We have to specify the 
# input shape of your inputs. And here, since we were working with colored images, therefore in three dimensions, corresponding 
# to the RGB code of colors. As, We actually resized in "Part 1 - Data Processing", are images down to 64 by 64. The input shape 
# of our images will be 64, 64 and 1.

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
# Adding the pooling layers to the convolutional layer

# MaxPool2D -> class
# pool_size -> the size of the frame
# strides=2 -> shift that frame every two pixels 


cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
# Adding a second convolutional layer with Max Pooling

# Remove the input_shape=[64, 64, 3] parameter because this one is entered only when adding the very first layer
# to automatically connect that first layer to the input layer, which automatically add the input layer

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
# Flatten -> class
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
# An one dimensional vector that will become the input of a fully connected neural network (a new layer)

#  unit -> the number of hidden neurons into this new fully connected layer
#  As long as it's not the final output layer, it is recommended to use a rectifier activation function (relu)

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
# Adding the final output layer, which will still be fully connected to that previous hidden layer

# Since We're doing binary classification, we only need one neuron to encode that binary class zero or one or, cat or dog
# Therefore, we only need one neuron.
# For the output layer, it is not recommended to have a rectifier activation function, but rather a sigmoid activation function
# activation='sigmoid' -> For binary classification
# activation='softmax' -> For doing multiclass classification

cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))